In [1]:
## Import necessary packages
# On Macs use osx
# For Windows use qt

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['animation.ffmpeg_path']=r'C:\Users\lhamr\OneDrive\Desktop\ffmpeg.exe'

from numpy.random import rand
from landscapeWithOcean import LandscapeWithOcean # Import methods from inside file landscapeWithOcean.py

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib qt

In [2]:
def AddHill(Z,NX,NY,xx,yy,r,h):

    for x in range(NX):
        for y in range(NY):
            dx = np.mod(x-xx+NX/2,NX)-NX/2; # difference i-i0 but apply p.b.c. 
            dy = np.mod(y-yy+NY/2,NY)-NY/2;
            dr = np.sqrt(dx**2+dy**2);
            if (dr<r):
                Z[x,y] += h * (np.cos(dr/r*np.pi/2.0))**2;

    return Z

In [3]:
### Define simulation grid and initial conditions

NX = 30*2 #number of rows | used to be 70*2
NY = 30*2 #number of columns | used to be 70*2
d  = 5 # grid spacing in meters
dx = d # keep dx=dy for simplicity
dy = d

LX=NX*dx
LY=NY*dy

# small random features in topography to begin erosion
n = NX
r = n//2 #VALUE OF R for circles 
Z = np.zeros((NX,NY))

for i in range(n): #fills a circle yellow
    for j in range(n):
        if (i-r)**2 + (j-r)**2 <= r**2: #(x-h)^2 + (y-k)^2 <= r^2 where (h,k) is origin
            Z[i,j] = 0.5
            
#copy previous but change value for A[i,j] from 1 to 0
for i in range(n//2): #vertical region from 0 to n//2
    for j in range(n): #horizontal region
        if i+j >= n//2 and j-i<=n//2: #left bound and right bound, longer region - smaller region <= n//2 to get downward diagonal bound
            Z[i,j] = 0.1
            xx = i
            yy = j
            r  = (0.5)*NX
            h  = (0.5)*1
            Z = AddHill(Z,NX,NY,xx,yy,r,h)
for i in range(n//2,n): #vertical region from n//2 to n
    for j in range(n):
        if i-j <= n//2 and j+i<=3*n//2: #think of what values of i and j is yellow (75+75)=150<=3*n//2
            Z[i,j] = 0.1
            xx = i
            yy = j
            r  = (0.5)*NX
            h  = (0.5)*1
            Z = AddHill(Z,NX,NY,xx,yy,r,h)

x = np.arange(NX)
y = np.arange(NY)
X,Y = np.meshgrid(y,x) #strange that y goes first !!!
ZMaxOrg = np.max(Z)
#print(ZMaxOrg)

In [4]:
### Physical Parameters
K = 1.0e-12 # meters^(1-2m)/yr | used to be 1.0e-6

D = 0.012 # m^2/yr | used to be 0.005

# uplift rate
# uplift = 0.03 / 600.
uplift = 0.0

In [5]:
### Model parameters

# Time step
dt = d**2 / D / 12. #used to be dt = d**2 / D / 8. 
#dt = d**2 / D /16. #extra small steps 
print(' dt[years] = ',dt)

#Area exponent A^m, default m=1
m=1.2

#gradient exponent g^n, default n=1
n=1.3

#erosion threshold 
theta_c = 10 

 dt[years] =  173.61111111111111


In [6]:
# Total simulation time
T = 1000.0 * 100.0

# total number of iterations
n_iter = int(np.round(T/dt))
print('Number of interation: ',n_iter)

Number of interation:  576


In [7]:
# Initialize landscape 
ls = LandscapeWithOcean(NX,NY)

oceanLevelParameter=0.5  # what does this parameter do?
ls.ComputeOceanVolumeFromOceanLevelParameter(Z,NX,NY,oceanLevelParameter)

ls.pool_check(Z,NX,NY)
ls.A = np.zeros((NX,NY))

Minimum elevation           0.6
Maximum elevation           220.64638403013174
Beach level                 110.62319201506587
Ocean volume                44776.107718719504
Percentage of ocean surface 30.5


In [8]:
# Set-up figure
def init_figure():
    fig = plt.figure(figsize=(12.,6.))
    plt.show()
    return fig

def update_figure():
        plt.clf()
        ax1 = fig.add_subplot(121,projection='3d')

        # use equal x-y aspect with an explicit vertical exageration
        vert_exag = 4.
        ax1.set_xlim3d(0,max(NX,NY))
        ax1.set_ylim3d(0,max(NX,NY))
        ax1.set_zlim3d(0,ZMaxOrg)

        ax1.set_title('Surface Relief')

#        surf = ax1.plot_surface(X,Y,Z, cmap = cm.terrain, rstride=1, cstride=1,
#                antialiased=False,linewidth=0)

        ZPlot = np.copy(Z)
        ZPlot[ZPlot<ls.ZBeachLevel] = ls.ZBeachLevel 
        ZPlot -= ls.ZBeachLevel
        ax1.plot_surface(X,Y,ZPlot, cmap = cm.terrain, rstride=1, cstride=1,
                antialiased=False,linewidth=0)

        ax2 = fig.add_subplot(122,aspect='equal')
        ax2.set_title('Elevation')

        #im = ax2.pcolor(Z,cmap=cm.terrain)
        im = ax2.pcolor(ZPlot,cmap=cm.coolwarm)
        cs = ax2.contour(ZPlot,6,colors='k')

        # Add a color bar which maps values to colors.
        cbar = fig.colorbar(im, shrink=0.5, aspect=5)
        # Add the contour line levels to the colorbar
        cbar.add_lines(cs)

        #plt.show()
        plt.draw()
        plt.pause(0.0001)

In [9]:
# Set up figure
# movie parameters
from matplotlib.animation import FFMpegWriter
metadata = dict(title='OCEAN',artist='laks',comment='creative')
writer = FFMpegWriter(fps=15, metadata=metadata,bitrate=200000)

fig = init_figure()
update_figure()
Znew = np.copy(Z)
    
with writer.saving(fig, "simulationparameterschanged.mp4", dpi=200):
    for it in range(1,n_iter+1):

        ls.calculate_collection_area(Z,NX,NY)
        ls.A *= dx*dy

        for i in range(NX):
            iL = np.mod(i-1,NX) # normally i-1 but observe p.b.c.
            iR = np.mod(i+1,NX) # normally i+1 but observe p.b.c.

            for j in range(NY):
                jD = np.mod(j-1,NY) # normally j-1 but observe p.b.c.
                jU = np.mod(j+1,NY) # normally j+1 but observe p.b.c.

                if ls.ocean[i,j]>0:
                    Psi_z  = 0;
                    Phi_z  = 0;
                else:
                    if ls.drain[i,j]>0: #this cell is a drain
                        s1 = (Z[iR,j]  - Z[iL,j] )/(2.*dx)
                        s2 = (Z[i,jU]  - Z[i,jD] )/(2.*dy)
                        s3 = (Z[iR,jD] - Z[iL,jU])/(2. * np.sqrt( dx**2 + dy**2) )
                        s4 = (Z[iR,jU] - Z[iL,jD])/(2. * np.sqrt( dx**2 + dy**2) )
                        gradient = (np.sqrt(s1**2 + s2**2) + np.sqrt(s3**2 + s4**2))/2.
                        Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)            
8
                    elif ls.drainage[i,j]>0: #this cell is a drainage point (it drains a pool)

                        if (Z[i,j]>=Z[iR,j]) and ls.pool[iR,j]!=ls.drainage[i,j]: 
                            gradient = (Z[i,j]-Z[iR,j])/dx #pool is on my left, I drain to the right, use this gradiant
                        elif (Z[i,j]>=Z[iL,j]) and ls.pool[iL,j]!=ls.drainage[i,j]:
                            gradient = (Z[i,j]-Z[iL,j])/dx
                        elif (Z[i,j]>=Z[i,jU]) and ls.pool[i,jU]!=ls.drainage[i,j]:
                            gradient = (Z[i,j]-Z[i,jU])/dy
                        elif (Z[i,j]>=Z[i,jD]) and ls.pool[i,jD]!=ls.drainage[i,j]:
                            gradient = (Z[i,j]-Z[i,jD])/dy
                        else:
                            gradient = 0.02 # ??? This does happen (maybe when two pools merge)
                        Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)

                    else: #this cell is a pool, assume it has some mass diffusion but no erosion!
                        Psi_z = 0.

                    if (Psi_z<0):
                        Psi_z = 0. 

                    # diffusion term
                    Phi_z = D * ( (Z[iR,j] - 2.*Z[i,j] + Z[iL,j]) / dx**2 \
                                + (Z[i,jU] - 2.*Z[i,j] + Z[i,jD]) / dx**2 )

                Znew[i,j] = Z[i,j] + (Phi_z - Psi_z + uplift )*dt  

                dZdt= (Znew[i,j] - Z[i,j]) / dt
                CFL = abs(dZdt) * dt / min(dx,dy)
                if (CFL>1.0):
                    print('\nWarning: Time step of',dt,'is probably too large. Safer would be:',dt/CFL)

                if (Znew[i,j]<0.):
                    Znew[i,j] = 0. # yes, this does happen at the boundary when kept at zero

        #Znew[0,:] = 0.0 # resets front boundary to 0
        Z = np.copy(Znew)

        ls.pool_check(Z,NX,NY)

        if (np.mod(it,10)==0): 
            print(it,end='')
            update_figure()
            writer.grab_frame()
            print(' Ocean level=',ls.ZBeachLevel,' Ocean surface fraction=',100*ls.AOcean/(NX*NY));
        else:
            print('.',end='')

    update_figure()
    writer.grab_frame()
    print(' Simulation finished.')

















































































































































.











































































































.





















































































































.




















































































































.


















































































































.










































































































.




































































.

























..10 Ocean level= 110.59741125949982  Ocean surface fraction= 31.305555555555557
.........20 Ocean level= 110.54161476768674  Ocean surface fraction= 31.916666666666668
.........30 Ocean lev

In [10]:
#(1)

#Describe in your own words, how the initial conditions are chosen and what geological features they represent.
#A: NX and NY are the number of rows and columns. d, dx, and dy are used in calculation of the rates of spacing. 
# Z is the initial set of random values that is generated for the height map of the array. xx, yy, r, and h are all random values mostly based on NX or NY. The values of Z are set to randomly generated hills. 
# They represent randomly generated hills as NX and NY varies, which causes the other variables to vary.

#Have a look inside landscapeWithOcean.py and give an interpretation of the variable ‘oceanLevelParameter’.  
#A: Since the landscape usually varied from a height from 0 to 1, the oceanLevelParameter was set to 0.1 by default. This meant values larger than 0.1 would not be submerged. Larger values of oceanLevelParameter means a higher ocean level and more submerged land.

#Run the code several times and describe the resulting evolution. Say how the geological features change with time. What happens in the ocean? 
#A: The resulting evolution shows the maximum height of the landscape decrease as erosion by water causes the height to decrease. The landscape becomes softened out. Hills and mountains become less sharp and pronounced. The ocean seems to envelope the land as much of the land submerges itself into the ocean over time from erosion.


In [11]:
#(2) 

#Describe what geological features you were trying to model even if their representation is not perfect.
#A: I was trying to model the effects of land mass presence in causing erosion as time goes by. From the simulation, we can see that there is a large blob of land connected to what seems like a box of thin barriers. The shape shows how the thin barriers, when not in near proximity to the large chunk of land, flattens out to allow a trough of water to flow through. However, the barriers that were connected to the large piece of land remained above water. Rather than being submerged, it was relatively smoothed out and showed how the ocean maneuvered around the land masses and caused some areas to submerge while others to remain above water.

#Briefly describe how your landscape evolved and identify effects that are missing from your simulation if there are any.
#A: As time in the simulation progresses, the barriers that were not connected to the large mass of land caved outwards as water eroded it away. The barriers connected to the large mass in the center remained higher than those that were not connected. The effect of the land mass being nearly submerged was not available due to the rate of erosion and time step not being changed to display this.


In [12]:
#(3)

#Effects the parameter changes had:
#K: The K value determines the zoom and perspective of the graph. Larger values of K were more zoomed out while smaller values were more zoomed in. This could be attributed to a faster rate of erosion.
#D: Larger values of D reduced the time step, which is important in monitoring the change through erosion. 
#n: The area exponent allows for larger areas of erosion to occur. Increases in this value showed the height and length changes.
#m: Higher values of the gradient exponent allowed for larger gradients when simulating. These gradients stretched out significantly more. 
#the ocean level: Higher values of the ocean level increases the ocean level. With all positive values, an ocean level of 0 would mean all of the land mass is above water.
#dt: The higher the time step, the more noticeable change through erosion was. 
#the spatial resolution: Increases in this allowed for better viewing through higher resolution.

#some interesting effects: The changes in water level allowed for the creation of some beautiful graphs as it becomes much easier to note height change differences from the simulation.